In [1]:
from dask.distributed import Client, as_completed
import numpy as np
import time
import os
from pyretis.setup import create_simulation
from pyretis.core.tis import shoot
from pyretis.core.retis import retis_swap_zero
from pyretis.inout.settings import parse_settings_file
# from infretis import REPEX_state, calc_cv_vector

In [2]:
# Set up simulation variables
INP = './retis.rst'
sim_settings = parse_settings_file(INP)
sim = create_simulation(sim_settings)
sim.set_up_output(sim_settings)
sim.initiate(sim_settings)


CREATING  ENSEMBLE  0
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  1
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  2
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  3
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  4
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".


CRITICAL:pyretis.inout.fileio:Could not open file "load/000/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^-] ends at the wrong interface!


Created engine "GROMACS engine" from settings.
Will initiate paths using method "load".

Loading data for path path ensemble [0^-]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/000/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^-]:
- Path with length 179 (max: None)
- Order parameter max: (0.270052, 0)
- Order parameter min: (0.263395, 89)
- Start 0.270052
- End 0.270052
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



CRITICAL:pyretis.inout.fileio:Could not open file "load/001/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^+] starts at the wrong interface!
CRITICAL:pyretis.inout.fileio:Could not open file "load/002/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [1^+] ends at the wrong interface!


# Results for path ensemble [0^-] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 179.
# Order parameter max was: 0.270052 at index 0.
# Order parameter min was: 0.263395 at index 89.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [0^-]         0           1           0           0           0           0           0


Loading data for path path ensemble [0^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/001/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^+]:
- Path with length 109 (max: None)
- Order parameter max: (0.279931, 54)
- Order parameter min: (0.26981, 0)
- Start 0.26981
- End 0.26981
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0

# Results for path ensemble [0^+] at cycle 0:
# 

CRITICAL:pyretis.inout.fileio:Could not open file "load/003/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [2^+] ends at the wrong interface!


# Results for path ensemble [1^+] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 291.
# Order parameter max was: 0.314728 at index 145.
# Order parameter min was: 0.269903 at index 0.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [1^+]         0           1           0           0           0           0           0


Loading data for path path ensemble [2^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/003/energy.txt
Energies are not set.
Sorting input files
Found initial path for [2^+]:
- Path with length 61 (max: None)
- Order parameter max: (0.307148, 30)
- Order parameter min: (0.269896, 0)
- Start 0.269896
- End 0.269896
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0

# Results for path ensemble [2^+] at cycle 0:

CRITICAL:pyretis.inout.fileio:Could not open file "load/004/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [3^+] ends at the wrong interface!


Loading data for path path ensemble [3^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/004/energy.txt
Energies are not set.
Sorting input files
Found initial path for [3^+]:
- Path with length 101 (max: None)
- Order parameter max: (0.336438, 50)
- Order parameter min: (0.269994, 0)
- Start 0.269994
- End 0.269994
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0

# Results for path ensemble [3^+] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 101.
# Order parameter max was: 0.336438 at index 50.
# Order parameter min was: 0.269994 at index 0.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [3^+]         0           1           0           0           0           0           0




True

In [7]:
def run_md(ens_num, input_traj, settings, ensembles, cycle):
    interfaces = settings['simulation']['interfaces']   
    out = {'ensembles': [],                                                  
           'accepted_trajs': [],                                        
           'traj_vectors': [],
           'status': None} 
    
    if len(ens_num) == 1:
        print('bling', ens_num)
        # ensembles[ens[0]]['path_ensemble'].last_path = input_traj
        start_cond = ensembles[ens_num[0]+1]['path_ensemble'].start_condition
        tis_settings = settings['ensemble'][ens_num[0]+1]['tis']  
        accept, trial, status = shoot(ensembles[ens_num[0]+1],
                                      tis_settings,
                                      start_cond)
        print(type(input_traj), type(trial), start_cond, 'baka!')
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj[0]
        if ens_num[0] < 0:
            interfaces = interfaces[0:1]
            print('cowboiiii', interfaces)
        cv_vector = calc_cv_vector(out_traj, interfaces)
        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = [out_traj]
        out['traj_vectors'] = [cv_vector]
        out['status'] = status

    else:
        print('blang')
        accept, trial, status = retis_swap_zero(ensembles, 
                                                settings,
                                                cycle)
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj
        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = out_traj
        ifaces = [[interfaces[0:1]], interfaces]
        for traj, iface in zip(out_traj, ifaces):
            out['traj_vectors'].append(calc_cv_vector(traj, iface))     
        out['status'] = status
    return out
        

In [4]:
# Connect to scheduler
from dask.distributed import Client, SSHCluster
n_workers = 2
client = Client("tcp://129.241.87.6:8786")
client

<Client: 'tcp://129.241.87.6:8786' processes=2 threads=2, memory=1.96 GiB>

In [8]:
#start some timing to keep track
walltime = 1*20 #(sec per hour)
interfaces = sim.settings['simulation']['interfaces']
size=len(interfaces)
start_time = time.time() 
current_time = start_time

# Setup the infretis simulation
state = REPEX_state(size, minus=True)
for i in range(size-1):
    # we add all the i+ paths.
    state.add_traj(ens=i, traj=sim.ensembles[i+1]['path_ensemble'].last_path.copy(),
                   valid=calc_cv_vector(sim.ensembles[i+1]['path_ensemble'].last_path, interfaces),
                   count=False)

# add minus path:
state.add_traj(ens=-1, traj=sim.ensembles[0]['path_ensemble'].last_path.copy(), valid=(1,), count=False)

In [51]:
# Define an iterator that is going to be our results queue
futures = as_completed(None, with_results=True)

# Start running
traj_i = 0

# submit the first batch of workers
for worker in range(n_workers):
    ens, input_traj = state.pick()
    
    # assign gromacs resurces to workers:
    for ens_num, traj_num in zip(ens, input_traj):
        ens_num += 1
        mdrun = sim.ensembles[ens_num]['engine'].mdrun.replace('0', f'{worker}')
        mdrun_c = sim.ensembles[ens_num]['engine'].mdrun_c.replace('0', f'{worker}')
        sim.ensembles[ens_num]['engine'].mdrun = mdrun
        sim.ensembles[ens_num]['engine'].mdrun_c = mdrun_c
        sim.ensembles[ens_num]['path_ensemble'].last_path = traj_num
        print(ens_num, traj_num.ordermax, sim.settings['ensemble'][ens_num]['simulation']['interfaces'], len(ens))
    
    fut = client.submit(run_md, ens, input_traj, sim.settings, sim.ensembles, traj_i, pure=False)
    futures.add(fut)
    traj_i += 1

4 (0.3367321371277769, 376) [0.27, 0.32, 0.6] 1
2 (0.2865969472231381, 83) [0.27, 0.28, 0.6] 1


In [52]:
for i in futures:
    output = i[1]
    ensembles = output['ensembles']
    out_trajs = output['accepted_trajs']
    traj_vectors = output['traj_vectors']
    for ens, out_traj, traj_v in zip(ensembles, out_trajs, traj_vectors):
        #ens+=1
        if ens >= 0 and state._offset != 0:
            valid0 = tuple([0 for _ in range(state._offset)] + list(traj_v))
        elif ens < 0:
            valid0 = tuple(list(traj_v) +
                          [0 for _ in range(state.n - state._offset)])

        print('------- pipipi 1-------')
        print(state.state[ens, :])
        print(valid0)
        print(ens)
        print('------- pipipi 1-------')
        state.add_traj(ens, out_traj, traj_v)
    print(i)

------- pipipi 1-------
[0. 1. 1. 0. 0. 0.]
(0, 1, 1, 0, 0, 0)
1
------- pipipi 1-------
(<Future: finished, type: dict, key: run_md-3426b0f5-19e0-44fb-a580-e2dc7f0cd541>, {'ensembles': [1], 'accepted_trajs': [<pyretis.core.path.Path object at 0x7f275c6078b0>], 'traj_vectors': [(1, 1, 0, 0, 0)], 'status': 'NCR'})
------- pipipi 1-------
[0. 1. 1. 1. 1. 0.]
(0, 1, 1, 1, 1, 0)
3
------- pipipi 1-------
(<Future: finished, type: dict, key: run_md-eb7befe2-fedc-47cd-86b8-f1aae23def39>, {'ensembles': [3], 'accepted_trajs': [<pyretis.core.path.Path object at 0x7f275c245f70>], 'traj_vectors': [(1, 1, 1, 1, 0)], 'status': 'NCR'})


In [53]:
for item, key in state.result.data.items():
    print(item, key)
# print(state.result.data)

-1 {(1.0, 0.0, 0.0, 0.0, 0.0, 0.0): 28.0}
0 {(0.0, 1.0, 0.0, 0.0, 0.0, 0.0): 17.0, (0.0, 1.0, 1.0, 0.0, 0.0, 0.0): 8.0}
1 {(0.0, 1.0, 1.0, 1.0, 0.0, 0.0): 13.0, (0.0, 1.0, 1.0, 0.0, 0.0, 0.0): 17.0}
2 {(0.0, 1.0, 1.0, 1.0, 0.0, 0.0): 20.0, (0.0, 1.0, 1.0, 1.0, 1.0, 0.0): 6.0}
3 {(0.0, 1.0, 1.0, 1.0, 1.0, 0.0): 27.0}


In [44]:
for item, key in state.result.data.items():
    print(item, key)
# print(state.result.data)

-1 {(1.0, 0.0, 0.0, 0.0, 0.0, 0.0): 20.0}
0 {(0.0, 1.0, 0.0, 0.0, 0.0, 0.0): 17.0, (0.0, 1.0, 1.0, 0.0, 0.0, 0.0): 1.0}
1 {(0.0, 1.0, 1.0, 1.0, 0.0, 0.0): 13.0, (0.0, 1.0, 1.0, 0.0, 0.0, 0.0): 8.0}
2 {(0.0, 1.0, 1.0, 1.0, 0.0, 0.0): 18.0}
3 {(0.0, 1.0, 1.0, 1.0, 1.0, 0.0): 20.0}


In [ ]:
# Start the main while loop:
while current_time < start_time+walltime:
    output = next(futures)[1]
    accept, paths, status = output
    traj_vectors = []
    if type(paths) == type(()):
        for path in paths:
            traj_vectors.append(calc_cv_vector(path, interfaces))
    else:
        traj_vectors.append(calc_cv_vector(path, interfaces))
    
    # for ens, out_traj, traj_v in zip(ensembles, out_trajs, traj_vectors):
    #     state.add_traj(ens, out_traj, traj_v)
    # if acc:    
    #     state.write_ensembles_daniel(steps_done)
    
    # steps_done += 1
    # ens, input_traj = state.pick() # pick a new shooting move
    # out_trajs = []
    # for _ in ens:
    #     out_trajs.append('something')
    #     traj_i += 1
    
    # fut = client.submit('bombimb')
    # futures.add(fut)
    
    # if current_time-start_time > 60*minutes:
    #     print(f"steps_done: {steps_done} in {current_time-start_time} seconds")
    #     minutes = (current_time-start_time)//60 + 1 
    # current_time = time.time()

In [38]:
# for future in futures:
#     output = future[1]
#     accept, paths, status = output
#     print(accept, paths, status)
#     print(type(accept), type(paths), type(status))
#     if type(paths) == type(()):
#         for path in paths:
#             sim.ensembles[0]['path_ensemble'].add_path_data(path, status)
#     else:
#         sim.ensembles[0]['path_ensemble'].add_path_data(paths, status)

False Path with length 1 (max: None)
Order parameter max: (0.27883328352260955, 0)
Order parameter min: (0.27883328352260955, 0)
Start 0.27883328352260955
End 0.27883328352260955
Status: Kicked outside of boundaries
Generated: Path was generated with a shooting move
Weight: 1.0 KOB
<class 'bool'> <class 'pyretis.core.path.Path'> <class 'str'>
True Path with length 617 (max: 100000)
Order parameter max: (0.3364439181973519, 367)
Order parameter min: (0.2698852458591557, 0)
Start 0.2698852458591557
End 0.269920569696948
Status: The path has been accepted
Generated: Path was generated with a shooting move
Weight: 1.0 ACC
<class 'bool'> <class 'pyretis.core.path.Path'> <class 'str'>


In [68]:
sim.settings['ensemble'][0]['tis']['maxlength']

{'freq': 0.5,
 'maxlength': 100000,
 'aimless': True,
 'allowmaxlength': False,
 'zero_momentum': False,
 'rescale_energy': False,
 'sigma_v': -1,
 'seed': 0,
 'high_accept': False,
 'rgen': 'rgen',
 'shooting_move': 'sh',
 'shooting_moves': [],
 'mirror_freq': 0,
 'target_freq': 0,
 'target_indices': [],
 'ensemble_number': 0,
 'detect': 0.28}

In [7]:
for i in futures:
    print(i)

(<Future: finished, type: tuple, key: retis_swap_zero-f9719b3e-3095-453b-a97f-c3d0cca5bce6>, (False, (<pyretis.core.path.Path object at 0x7f1fa31a0d90>, <pyretis.core.path.Path object at 0x7f1fb07952e0>), 'BTS'))
(<Future: finished, type: tuple, key: shoot-2c21c319-f846-43b7-ba25-a2f99454c070>, (True, <pyretis.core.path.Path object at 0x7f1fa31a06d0>, 'ACC'))


In [51]:
t_time = 0 # record total MD time (time spend by the workers)
minutes = 0
steps_done = 0
while current_time < start_time+walltime:
    output = next(futures)[1]
    accept, out_path, status = output
    
    # add_path_data

    # add traj to state which unlocks it
    
    # if acc, write ensembles ...
    
    # submit and add future
    

(<Future: finished, type: tuple, key: shoot-c2e6833f-9518-4e31-82c8-69bf95c855bb>, (True, <pyretis.core.path.Path object at 0x7f8b504ae0a0>, 'ACC'))
(<Future: finished, type: tuple, key: shoot-528337f6-c50c-4120-a8ca-8f8eb81d4f4f>, (True, <pyretis.core.path.Path object at 0x7f8b5068fc10>, 'ACC'))


2022-09-20 21:02:01,974 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [85]:
futures = as_completed(None, with_results=True)

for i in range(2):
    mdrun = sim.ensembles[i]['engine'].mdrun.replace('0', f'{i}')
    mdrun_c = sim.ensembles[i]['engine'].mdrun_c.replace('0', f'{i}')
    sim.ensembles[i]['engine'].mdrun = mdrun
    sim.ensembles[i]['engine'].mdrun_c = mdrun_c
    j = client.submit(shoot, 
                      sim.ensembles[i], 
                      sim.settings['ensemble'][i]['tis'],
                      sim.ensembles[i]['path_ensemble'].start_condition,
                      pure=False)
    time.sleep(5)
    futures.add(j)
    
for i in futures:
    print(i)

(<Future: finished, type: tuple, key: shoot-cb4069e6-4131-4931-a0bb-ccb66307d9e1>, (False, <pyretis.core.path.Path object at 0x7fc8cc2ce280>, 'KOB'))
(<Future: finished, type: tuple, key: shoot-c28eceb5-322c-4349-a918-e09372c4ef41>, (False, <pyretis.core.path.Path object at 0x7fc8cc265fd0>, 'BTL'))


In [45]:
print(dir(sim))
print(sim.settings['simulation']['interfaces'])

os.getcwd()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_task', 'create_output_tasks', 'cycle', 'ensembles', 'exe_dir', 'execute_tasks', 'extend_cycles', 'first_step', 'initiate', 'is_finished', 'load_restart_info', 'name', 'output_tasks', 'required_settings', 'restart_freq', 'restart_info', 'rgen', 'run', 'set_up_output', 'settings', 'simulation_output', 'simulation_type', 'soft_exit', 'step', 'system', 'tasks', 'write_restart']
[0.5, 0.55, 0.6, 0.65, 0.7]


'/home/daniel/e-dask/b-salt'

In [82]:
# Define some infRETIS infrastructure
def calc_cv_vector(path, interfaces):                                           
    path_max, _ = path.ordermax                                                 
    value = 1                                                                   
    return tuple(value if i <= path_max else 0 for i in interfaces)   

def calculate_ensemble_prob_and_weight(ens, num, offset=0):
    keys = np.array(list(ens.keys()))
    values = np.array(list(ens.values()))
    results = np.zeros(len(keys[0]), dtype="float128")
    idx = np.argsort(values)
    for key, value in zip(keys[idx], values[idx]):
        # 1/key[num] in the following line is the MC reweighting
        results += (np.array([1 if i else 0 for i in key]) *
                    value/abs(key[num+offset]))
    if num < 0:
        offset -= 1
    if results[offset] == 0:
        raise ValueError(f"{ens},{num},{offset},{results}")

    return results/results[offset], results[offset]


def analyze_data(data, offset=0):
    total_prob = np.array([1.0], dtype="float128")
    overall_prob = [1.0]
    ensembles = np.sort(list(data.data.keys()))
    for ens in ensembles:
        if ens < 0:
            continue
        prob, _ = calculate_ensemble_prob_and_weight(data.data[ens], ens,
                                                     offset=offset)
        if ens+offset+1 >= len(prob):
            total_prob = 0
            overall_prob.append(0)
            break
        next_prob = prob[ens+offset+1]
        overall_prob.append(next_prob*total_prob[0])
        total_prob *= next_prob  # cross next interface
    return overall_prob, total_prob


class Results(object):
    def __init__(self, n=3, offset=0):
        self.data = {}
        self.avg_lengths = {}
        self._run_prob = {}
        self._run_weight = {}
        self._cycles = {}
        self._run_total_prob = []
        self._n = n
        self._offset = offset

    def update_ens(self, ens, traj, weight, length=None):
        e = self.data.get(ens, {})
        w = e.get(traj, 0)
        e[traj] = w+weight
        self.data[ens] = e
        if length is not None and weight != 0:
            # update pathlengths for flux calculation
            avg, total_weight = self.avg_lengths.get(ens, (0, 0))
            total_weight += weight
            avg += weight*(length-avg)/total_weight
            self.avg_lengths[ens] = (avg, total_weight)

    def update_run_prob(self, ens, n=0):
        run_p = self._run_prob.get(ens, [])
        run_w = self._run_weight.get(ens, [])
        run_c = self._cycles.get(ens, [])
        data = self.data.get(ens, None)
        if data is not None:
            temp = calculate_ensemble_prob_and_weight(data, ens,
                                                      offset=self._offset)

            prob, tot_weight = temp
            run_p.append(prob[ens+self._offset+1])
            run_w.append(tot_weight)
            run_c.append(n)
        self._run_prob[ens] = run_p
        self._run_weight[ens] = run_w
        self._cycles[ens] = run_c

    def update_run_total_prob(self):
        _, tot = analyze_data(self, self._offset)
        self._run_total_prob.append(tot)


class REPEX_state(object):
    def __init__(self, n=3, result=None, minus=False):
        if minus:
            self._offset = int(minus)
            n += int(minus)
        else:
            self._offset = 0

        self.n = n
        self.state = np.zeros(shape=(n, n))
        self._locks = np.ones(shape=(n))
        self._last_prob = None
        self._random_count = 0
        self.result = Results(n, offset=self._offset)
        self._n = 0
        self._trajs = ["" for i in range(n)]

    def pick(self):
        prob = self.prob.astype("float64").flatten()
        p = np.random.choice(self.n**2, p=np.nan_to_num(prob/np.sum(prob)))
        traj, ens = np.divmod(p, self.n)
        self.swap(traj, ens)
        self.lock(ens)
        traj = self._trajs[ens]
        # If available do 0+- swap with 50% probability
        if ((
             (ens == self._offset and not self._locks[self._offset-1]) or
             (ens == self._offset-1 and not self._locks[self._offset])
        ) and np.random.random() < 0.5):
            if ens == self._offset:
                # ens = 0
                other = self._offset - 1
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (other_traj, traj)
            else:
                # ens = -1
                other = self._offset
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (traj, other_traj)
        return (ens-self._offset,), (traj,)

    def pick_traj_ens(self, ens):
        prob = self.prob.astype("float64")[:, ens].flatten()
        traj = np.random.choice(self.n, p=np.nan_to_num(prob/np.sum(prob)))
        self.swap(traj, ens)
        self.lock(ens)
        return self._trajs[ens]

    def write_ensembles(self):
        out = self.prob
        out = out.T
        out = np.nan_to_num(out)
        for i, ens in enumerate(out):
            if self._locks[i]:
                continue
            for j, weight in enumerate(ens):
                if weight != 0:
                    traj = self.state[j]
                    path = self._trajs[j]
                    if i in [self._offset, self._offset-1]:
                        # ens 0 and -1
                        length = getattr(path, 'length', None)
                    else:
                        length = None
                    self.result.update_ens(i-self._offset, tuple(traj), weight,
                                           length=length)
            self.result.update_run_prob(i-self._offset, n=self._n)
        self.result.update_run_total_prob()

    def add_traj(self, ens, traj, valid, count=True, n=0):
        print(ens, valid, 'bing0')
        if ens >= 0 and self._offset != 0:
            valid = tuple([0 for _ in range(self._offset)] + list(valid))
        elif ens < 0:
            valid = tuple(list(valid) +
                          [0 for _ in range(self.n - self._offset)])
        ens += self._offset
        assert valid[ens] != 0
        # invalidate last prob
        self._last_prob = None
        self._trajs[ens] = traj
        print(ens, valid, 'bing1')
        self.state[ens, :] = valid
        self.unlock(ens)
        if count:
            self.write_ensembles()
            self._n += 1

    def lock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 0
        self._locks[ens] = 1

    def unlock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 1
        self._locks[ens] = 0

    def swap(self, traj, ens):
        # mainly to keep the locks symmetric
        self.state[[ens, traj]] = self.state[[traj, ens]].copy()
        temp1 = self._trajs[ens]
        self._trajs[ens] = self._trajs[traj]
        self._trajs[traj] = temp1

    @property
    def prob(self):
        if self._last_prob is None:
            prob = self.inf_retis(abs(self.state), self._locks)
            self._last_prob = prob.copy()
        return self._last_prob

    def inf_retis(self, input_mat, locks):
        # Drop locked rows and columns
        bool_locks = locks == 1
        # get non_locked minus interfaces
        offset = self._offset - sum(bool_locks[:self._offset])
        # make insert list
        i = 0
        insert_list = []
        for lock in bool_locks:
            if lock:
                insert_list.append(i)
            else:
                i += 1

        # Drop locked rows and columns
        non_locked = input_mat[~bool_locks, :][:, ~bool_locks]

        # Sort based on the index of the last non-zero values in the rows
        # argmax(a>0) gives back the first column index that is nonzero
        # so looping over the columns backwards and multiplying by -1 gives the
        # right ordering
        minus_idx = np.argsort(np.argmax(non_locked[:offset] > 0, axis=1))
        pos_idx = (np.argsort(-1 *
                              np.argmax(non_locked[offset:, ::-1] > 0, axis=1)
                              )
                   + offset)

        sort_idx = np.append(minus_idx, pos_idx)
        sorted_non_locked = non_locked[sort_idx]

        # check if all trajectories have equal weights
        sorted_non_locked_T = sorted_non_locked.T
        # Check the minus interfaces
        equal_minus = np.all(sorted_non_locked_T[
            np.where(sorted_non_locked_T[:, :offset] !=
                     sorted_non_locked_T[offset-1, :offset])
            ] == 0)
        # check the positive interfaces
        if len(sorted_non_locked_T) <= offset:
            equal_pos = True
        else:
            equal_pos = np.all(sorted_non_locked_T[:, offset:][
                np.where(sorted_non_locked_T[:, offset:] !=
                         sorted_non_locked_T[offset, offset:])
                ] == 0)

        equal = equal_minus and equal_pos

        out = np.zeros(shape=sorted_non_locked.shape, dtype="float128")
        if equal:
            # All trajectories have equal weights, run fast algorithm
            # run_fast
            # minus move should be run backwards
            out[:offset, ::-1] = self.quick_prob(sorted_non_locked[:offset,
                                                                   ::-1])
            if offset < len(out):
                # Catch only minus ens available
                out[offset:] = self.quick_prob(sorted_non_locked[offset:])
        else:
            #TODO DEBUG print
            print("DEBUG this should not happen outside of wirefencing")
            blocks = self.find_blocks(sorted_non_locked, offset=offset)
            for start, stop, direction in blocks:
                if direction == -1:
                    cstart, cstop = stop-1, start-1
                    if cstop < 0:
                        cstop = None
                else:
                    cstart, cstop = start, stop
                subarr = sorted_non_locked[start:stop, cstart:cstop:direction]
                subarr_T = subarr.T
                if len(subarr) == 1:
                    out[start:stop, start:stop] = 1
                elif np.all(subarr_T[np.where(subarr_T != subarr_T[0])] == 0):
                    # Either the same weight as the last one or zero
                    temp = self.quick_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                elif len(subarr) <= 12:
                    # We can run this subsecond
                    temp = self.permanent_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                else:
                    self._random_count += 1
                    #TODO DEBUG PRINTS
                    print(f"random #{self._random_count}, "
                          f"dims = {len(subarr)}")
                    # do n random parrallel samples
                    temp = self.random_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp

        out[sort_idx] = out.copy()  # COPY REQUIRED TO NOT BRAKE STATE!!!

        # Make sure we have a valid probability square
        assert np.allclose(np.sum(out, axis=1), 1)
        assert np.allclose(np.sum(out, axis=0), 1)

        # reinsert zeroes for the locked ensembles
        final_out_rows = np.insert(out, insert_list, 0, axis=0)

        # reinsert zeroes for the locked trajectories
        final_out = np.insert(final_out_rows, insert_list, 0, axis=1)

        return final_out

    def find_blocks(self, arr, offset):
        if len(arr) == 1:
            return ((0, 1, 1))
        # Assume no zeroes on the diagonal or lower triangle
        temp_arr = arr.copy()
        # for counting minus blocks
        temp_arr[:offset, :offset] = arr[:offset, :offset].T
        temp_arr[offset:, :offset] = 1  # add ones to the lower triangle
        non_zero = np.count_nonzero(temp_arr, axis=1)
        blocks = []
        start = 0
        for i, e in enumerate(non_zero):
            if e == i + 1:
                direction = -1 if start < offset else 1
                blocks.append((start, e, direction))
                start = e
        return blocks

    def quick_prob(self, arr):
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')
        working_mat = np.where(arr != 0, 1, 0)  # convert non-zero numbers to 1

        for i, column in enumerate(working_mat.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def force_quick_prob(self, arr):
        # TODO: DEBUG CODE
        # ONLY HERE TO DEBUG THE OTHER MEHTODS
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')

        force_arr = arr.copy()
        # Force everything to be identical
        force_arr[np.where(force_arr != 0)] = 1
        for i, column in enumerate(force_arr.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def permanent_prob(self, arr):
        out = np.zeros(shape=arr.shape, dtype="float128")
        n = len(arr)
        for i in range(n):
            rows = [r for r in range(n) if r != i]
            sub_arr = arr[rows, :]
            for j in range(n):
                if arr[i][j] == 0:
                    continue
                columns = [r for r in range(n) if r != j]
                M = sub_arr[:, columns]
                f = self.fast_glynn_perm(M)
                out[i][j] = f*arr[i][j]
        return out/max(np.sum(out, axis=1))

    def random_prob(self, arr, n=10_000):
        out = np.eye(len(arr), dtype="float128")
        current_state = np.eye(len(arr))
        choices = len(arr)//2
        even = choices*2 == len(arr)

        # The probability to go right
        prob_right = np.nan_to_num(np.roll(arr, -1, axis=1)/arr)

        # The probability to go left
        prob_left = np.nan_to_num(np.roll(arr, 1, axis=1)/arr)

        start = 0
        zero_one = np.array([0, 1])
        p_m = np.array([1, -1])
        temp = np.where(current_state == 1)

        for i in range(n):
            direction = np.random.choice(p_m)
            if not even:
                start = np.random.choice(zero_one)

            temp_left = prob_left[temp]
            temp_right = prob_right[temp]

            if not even:
                start = np.random.choice(zero_one)

            if direction == -1:
                probs = (temp_left[start:-1:2] *
                         np.roll(temp_right, 1, axis=0)[start:-1:2])
            else:
                probs = temp_right[start:-1:2]*temp_left[start+1::2]

            r_nums = np.random.random(choices)
            success = r_nums < probs

            for j in np.where(success)[0]:
                idx = j*2+start
                temp_state = current_state[:, [idx+direction, idx]]
                current_state[:, [idx, idx+direction]] = temp_state
                temp_state_2 = temp[0][[idx+direction, idx]]
                temp[0][[idx, idx+direction]] = temp_state_2

            out += current_state

        return out/(n+1)

    def fast_glynn_perm(self, M):
        def cmp(a, b):
            if a == b:
                return 0
            elif a > b:
                return 1
            else:
                return -1
        row_comb = np.sum(M, axis=0, dtype='float128')
        n = len(M)

        total = 0
        old_grey = 0
        sign = +1

        binary_power_dict = {2**i: i for i in range(n)}
        num_loops = 2**(n-1)

        for bin_index in range(1, num_loops + 1):
            total += sign * np.multiply.reduce(row_comb)

            new_grey = bin_index ^ (bin_index // 2)
            grey_diff = old_grey ^ new_grey
            grey_diff_index = binary_power_dict[grey_diff]
            direction = 2 * cmp(old_grey, new_grey)
            if direction:
                new_vector = M[grey_diff_index]
                row_comb += new_vector * direction

            sign = -sign
            old_grey = new_grey

        return total//num_loops

    def write_ensembles_daniel(self, step):
        out = self.prob
        out = out.T
        out = np.nan_to_num(out)
        
        for i, ens in enumerate(out):
            if self._locks[i]:
                continue
                
            with open(f'./{i}-pathensemble.txt', 'a') as writer:
                print(f'-- ensemble: {i} --')
                writer.write(f'{step}\t{sum([ww != 0. for ww in ens])}\n')    
                for j, weight in enumerate(ens):
                    if weight != 0:
                        traj = self.state[j]
                        path = self._trajs[j]
                        if i in [self._offset, self._offset-1]:
                            # ens 0 and -1
                            length = getattr(path, 'length', None)
                        else:
                            length = None
                        R = 'R' if traj[-1] == 1 else 'L'
                        min_o, max_o = path.ordermin[0], path.ordermax[0]
                        min_idx, max_idx = path.ordermin[1], path.ordermax[1]
                        MC = path.get_move()
                        if MC == None:
                            MC = 'ki'
                        # print(step, weight, 'L', 'R', MC, min_o, max_o, min_idx, max_idx)
                        # print(f'{step}\t{weight:.02f}\tL {R}\t{MC}\t{min_o:.02f}\t{max_o:.02f}')
                        # step weight L R length MC min-O max-O idx-min idx-max
                        # f'{step}\t\t{L}\t\{R}\t\t{}'
                        writer.write(f'{step}\t{weight:.02f}\tL {R}\t{MC}\t{min_o:.02f}\t{max_o:.02f}\t{path.number}\n')    
                        # file write something here...

                        # self.result.update_ens(i-self._offset, tuple(traj), weight,
                        #                        length=length)
                # self.result.update_run_prob(i-self._offset, n=self._n)
            # self.result.update_run_total_prob() 

In [105]:
#start some timing to keep track
walltime = 1*20 #(sec per hour)
interfaces = sim.settings['simulation']['interfaces']
size=len(interfaces)
print(size)
start_time = time.time() 
current_time = start_time

# Setup the infretis simulation
state = REPEX_state(size, minus=True)
offset = state._offset
print(offset)
for i in range(size-1):
    state.add_traj(ens=i,
                   traj=sim.ensembles[i]['path_ensemble'].last_path.copy(),
                   valid=calc_cv_vector(sim.ensembles[i]['path_ensemble'].last_path.copy(), interfaces),
                   count=False)
    print(i, 'ow')
# lock down last ens (as that one alsways enters the state)
initial_vector = calc_cv_vector(sim.ensembles[-1]['path_ensemble'].last_path.copy(), interfaces)                       
state.add_traj(ens=size-1, traj=sim.ensembles[-1]['path_ensemble'].last_path.copy(), valid=tuple(1 for _ in initial_vector), count=False)
state.lock(size)

# Define an iterator that is going to be our results queue
futures = as_completed(None, with_results=True)

# Start running
traj_i = 0
ensemble_sel = [1,2,3,4]
for _ in range(n_workers):
    ens = np.random.choice(ensemble_sel)
    ensemble_sel.pop(ensemble_sel.index(ens))
    print('ensens', ens)
    mdrun = sim.ensembles[ens]['engine'].mdrun.replace('0', f'{i}')
    mdrun_c = sim.ensembles[ens]['engine'].mdrun_c.replace('0', f'{i}')
    sim.ensembles[ens]['engine'].mdrun = mdrun
    sim.ensembles[ens]['engine'].mdrun_c = mdrun_c
    fut = client.submit(shoot, 
                        sim.ensembles[ens], 
                        sim.settings['ensemble'][ens]['tis'],
                        sim.ensembles[ens]['path_ensemble'].start_condition,
                        pure=False)
    futures.add(fut)
    traj_i += 1
    time.sleep(5)
    
for i in futures:
    print(i)

t_time = 0 # record total MD time (time spend by the workers)
minutes = 0
steps_done = 0
# while current_time < start_time+walltime:


5
1
0 (1, 1, 1, 1, 1) bing0
1 (0, 1, 1, 1, 1, 1) bing1
0 ow
1 (1, 1, 1, 1, 1) bing0
2 (0, 1, 1, 1, 1, 1) bing1
1 ow
2 (1, 1, 1, 1, 1) bing0
3 (0, 1, 1, 1, 1, 1) bing1
2 ow
3 (1, 1, 1, 1, 1) bing0
4 (0, 1, 1, 1, 1, 1) bing1
3 ow
4 (1, 1, 1, 1, 1) bing0
5 (0, 1, 1, 1, 1, 1) bing1
ensens 4
ensens 1
(<Future: finished, type: tuple, key: shoot-af813cfa-cfbc-4fe1-81f9-761b537ef439>, (False, <pyretis.core.path.Path object at 0x7fc8bdff1a60>, 'BTL'))
(<Future: finished, type: tuple, key: shoot-d9859edd-fc6b-4eb1-bedf-c40b33936fde>, (False, <pyretis.core.path.Path object at 0x7fc8c73e8d00>, 'BTL'))


In [53]:
sim.ensembles[i]['path_ensemble'].last_path.copy()

In [99]:
print(np.random.choice([1,2,3,4]))
print(interfaces)

3
[0.5, 0.55, 0.6, 0.65, 0.7]


In [5]:
# Define some infRETIS infrastructure
def calculate_ensemble_prob_and_weight(ens, num, offset=0):
    keys = np.array(list(ens.keys()))
    values = np.array(list(ens.values()))
    results = np.zeros(len(keys[0]), dtype="float128")
    idx = np.argsort(values)
    for key, value in zip(keys[idx], values[idx]):
        # 1/key[num] in the following line is the MC reweighting
        results += (np.array([1 if i else 0 for i in key]) *
                    value/abs(key[num+offset]))
    if num < 0:
        offset -= 1
    if results[offset] == 0:
        raise ValueError(f"{ens},{num},{offset},{results}")

    return results/results[offset], results[offset]


def analyze_data(data, offset=0):
    total_prob = np.array([1.0], dtype="float128")
    overall_prob = [1.0]
    ensembles = np.sort(list(data.data.keys()))
    for ens in ensembles:
        if ens < 0:
            continue
        prob, _ = calculate_ensemble_prob_and_weight(data.data[ens], ens,
                                                     offset=offset)
        if ens+offset+1 >= len(prob):
            total_prob = 0
            overall_prob.append(0)
            break
        next_prob = prob[ens+offset+1]
        overall_prob.append(next_prob*total_prob[0])
        total_prob *= next_prob  # cross next interface
    return overall_prob, total_prob


class Results(object):
    def __init__(self, n=3, offset=0):
        self.data = {}
        self.avg_lengths = {}
        self._run_prob = {}
        self._run_weight = {}
        self._cycles = {}
        self._run_total_prob = []
        self._n = n
        self._offset = offset

    def update_ens(self, ens, traj, weight, length=None):
        e = self.data.get(ens, {})
        w = e.get(traj, 0)
        e[traj] = w+weight
        self.data[ens] = e
        if length is not None and weight != 0:
            # update pathlengths for flux calculation
            avg, total_weight = self.avg_lengths.get(ens, (0, 0))
            total_weight += weight
            avg += weight*(length-avg)/total_weight
            self.avg_lengths[ens] = (avg, total_weight)

    def update_run_prob(self, ens, n=0):
        run_p = self._run_prob.get(ens, [])
        run_w = self._run_weight.get(ens, [])
        run_c = self._cycles.get(ens, [])
        data = self.data.get(ens, None)
        if data is not None:
            temp = calculate_ensemble_prob_and_weight(data, ens,
                                                      offset=self._offset)

            prob, tot_weight = temp
            run_p.append(prob[ens+self._offset+1])
            run_w.append(tot_weight)
            run_c.append(n)
        self._run_prob[ens] = run_p
        self._run_weight[ens] = run_w
        self._cycles[ens] = run_c

    def update_run_total_prob(self):
        _, tot = analyze_data(self, self._offset)
        self._run_total_prob.append(tot)


class REPEX_state(object):
    def __init__(self, n=3, result=None, minus=False):
        if minus:
            self._offset = int(minus)
            n += int(minus)
        else:
            self._offset = 0

        self.n = n
        self.state = np.zeros(shape=(n, n))
        self._locks = np.ones(shape=(n))
        self._last_prob = None
        self._random_count = 0
        self.result = Results(n, offset=self._offset)
        self._n = 0
        self._trajs = ["" for i in range(n)]

    def pick(self):
        prob = self.prob.astype("float64").flatten()
        p = np.random.choice(self.n**2, p=np.nan_to_num(prob/np.sum(prob)))
        traj, ens = np.divmod(p, self.n)
        self.swap(traj, ens)
        self.lock(ens)
        traj = self._trajs[ens]
        # If available do 0+- swap with 50% probability
        if ((
             (ens == self._offset and not self._locks[self._offset-1]) or
             (ens == self._offset-1 and not self._locks[self._offset])
        ) and np.random.random() < 0.5):
            if ens == self._offset:
                # ens = 0
                other = self._offset - 1
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (other_traj, traj)
            else:
                # ens = -1
                other = self._offset
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (traj, other_traj)
        return (ens-self._offset,), (traj,)

    def pick_traj_ens(self, ens):
        prob = self.prob.astype("float64")[:, ens].flatten()
        traj = np.random.choice(self.n, p=np.nan_to_num(prob/np.sum(prob)))
        self.swap(traj, ens)
        self.lock(ens)
        return self._trajs[ens]

    def write_ensembles(self):
        out = self.prob
        out = out.T
        out = np.nan_to_num(out)
        for i, ens in enumerate(out):
            if self._locks[i]:
                continue
            for j, weight in enumerate(ens):
                if weight != 0:
                    traj = self.state[j]
                    path = self._trajs[j]
                    if i in [self._offset, self._offset-1]:
                        # ens 0 and -1
                        length = getattr(path, 'length', None)
                    else:
                        length = None
                    self.result.update_ens(i-self._offset, tuple(traj), weight,
                                           length=length)
            self.result.update_run_prob(i-self._offset, n=self._n)
        self.result.update_run_total_prob()

    def add_traj(self, ens, traj, valid, count=True, n=0):
        if ens >= 0 and self._offset != 0:
            valid = tuple([0 for _ in range(self._offset)] + list(valid))
        elif ens < 0:
            valid = tuple(list(valid) +
                          [0 for _ in range(self.n - self._offset)])
        ens += self._offset
        assert valid[ens] != 0
        # invalidate last prob
        self._last_prob = None
        self._trajs[ens] = traj
        self.state[ens, :] = valid
        self.unlock(ens)
        if count:
            self.write_ensembles()
            self._n += 1

    def lock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 0
        self._locks[ens] = 1

    def unlock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 1
        self._locks[ens] = 0

    def swap(self, traj, ens):
        # mainly to keep the locks symmetric
        self.state[[ens, traj]] = self.state[[traj, ens]].copy()
        temp1 = self._trajs[ens]
        self._trajs[ens] = self._trajs[traj]
        self._trajs[traj] = temp1

    @property
    def prob(self):
        if self._last_prob is None:
            prob = self.inf_retis(abs(self.state), self._locks)
            self._last_prob = prob.copy()
        return self._last_prob

    def inf_retis(self, input_mat, locks):
        # Drop locked rows and columns
        bool_locks = locks == 1
        # get non_locked minus interfaces
        offset = self._offset - sum(bool_locks[:self._offset])
        # make insert list
        i = 0
        insert_list = []
        for lock in bool_locks:
            if lock:
                insert_list.append(i)
            else:
                i += 1

        # Drop locked rows and columns
        non_locked = input_mat[~bool_locks, :][:, ~bool_locks]

        # Sort based on the index of the last non-zero values in the rows
        # argmax(a>0) gives back the first column index that is nonzero
        # so looping over the columns backwards and multiplying by -1 gives the
        # right ordering
        minus_idx = np.argsort(np.argmax(non_locked[:offset] > 0, axis=1))
        pos_idx = (np.argsort(-1 *
                              np.argmax(non_locked[offset:, ::-1] > 0, axis=1)
                              )
                   + offset)

        sort_idx = np.append(minus_idx, pos_idx)
        sorted_non_locked = non_locked[sort_idx]

        # check if all trajectories have equal weights
        sorted_non_locked_T = sorted_non_locked.T
        # Check the minus interfaces
        equal_minus = np.all(sorted_non_locked_T[
            np.where(sorted_non_locked_T[:, :offset] !=
                     sorted_non_locked_T[offset-1, :offset])
            ] == 0)
        # check the positive interfaces
        if len(sorted_non_locked_T) <= offset:
            equal_pos = True
        else:
            equal_pos = np.all(sorted_non_locked_T[:, offset:][
                np.where(sorted_non_locked_T[:, offset:] !=
                         sorted_non_locked_T[offset, offset:])
                ] == 0)

        equal = equal_minus and equal_pos

        out = np.zeros(shape=sorted_non_locked.shape, dtype="float128")
        if equal:
            # All trajectories have equal weights, run fast algorithm
            # run_fast
            # minus move should be run backwards
            out[:offset, ::-1] = self.quick_prob(sorted_non_locked[:offset,
                                                                   ::-1])
            if offset < len(out):
                # Catch only minus ens available
                out[offset:] = self.quick_prob(sorted_non_locked[offset:])
        else:
            #TODO DEBUG print
            print("DEBUG this should not happen outside of wirefencing")
            blocks = self.find_blocks(sorted_non_locked, offset=offset)
            for start, stop, direction in blocks:
                if direction == -1:
                    cstart, cstop = stop-1, start-1
                    if cstop < 0:
                        cstop = None
                else:
                    cstart, cstop = start, stop
                subarr = sorted_non_locked[start:stop, cstart:cstop:direction]
                subarr_T = subarr.T
                if len(subarr) == 1:
                    out[start:stop, start:stop] = 1
                elif np.all(subarr_T[np.where(subarr_T != subarr_T[0])] == 0):
                    # Either the same weight as the last one or zero
                    temp = self.quick_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                elif len(subarr) <= 12:
                    # We can run this subsecond
                    temp = self.permanent_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                else:
                    self._random_count += 1
                    #TODO DEBUG PRINTS
                    print(f"random #{self._random_count}, "
                          f"dims = {len(subarr)}")
                    # do n random parrallel samples
                    temp = self.random_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp

        out[sort_idx] = out.copy()  # COPY REQUIRED TO NOT BRAKE STATE!!!

        # Make sure we have a valid probability square
        assert np.allclose(np.sum(out, axis=1), 1)
        assert np.allclose(np.sum(out, axis=0), 1)

        # reinsert zeroes for the locked ensembles
        final_out_rows = np.insert(out, insert_list, 0, axis=0)

        # reinsert zeroes for the locked trajectories
        final_out = np.insert(final_out_rows, insert_list, 0, axis=1)

        return final_out

    def find_blocks(self, arr, offset):
        if len(arr) == 1:
            return ((0, 1, 1))
        # Assume no zeroes on the diagonal or lower triangle
        temp_arr = arr.copy()
        # for counting minus blocks
        temp_arr[:offset, :offset] = arr[:offset, :offset].T
        temp_arr[offset:, :offset] = 1  # add ones to the lower triangle
        non_zero = np.count_nonzero(temp_arr, axis=1)
        blocks = []
        start = 0
        for i, e in enumerate(non_zero):
            if e == i + 1:
                direction = -1 if start < offset else 1
                blocks.append((start, e, direction))
                start = e
        return blocks

    def quick_prob(self, arr):
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')
        working_mat = np.where(arr != 0, 1, 0)  # convert non-zero numbers to 1

        for i, column in enumerate(working_mat.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def force_quick_prob(self, arr):
        # TODO: DEBUG CODE
        # ONLY HERE TO DEBUG THE OTHER MEHTODS
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')

        force_arr = arr.copy()
        # Force everything to be identical
        force_arr[np.where(force_arr != 0)] = 1
        for i, column in enumerate(force_arr.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def permanent_prob(self, arr):
        out = np.zeros(shape=arr.shape, dtype="float128")
        n = len(arr)
        for i in range(n):
            rows = [r for r in range(n) if r != i]
            sub_arr = arr[rows, :]
            for j in range(n):
                if arr[i][j] == 0:
                    continue
                columns = [r for r in range(n) if r != j]
                M = sub_arr[:, columns]
                f = self.fast_glynn_perm(M)
                out[i][j] = f*arr[i][j]
        return out/max(np.sum(out, axis=1))

    def random_prob(self, arr, n=10_000):
        out = np.eye(len(arr), dtype="float128")
        current_state = np.eye(len(arr))
        choices = len(arr)//2
        even = choices*2 == len(arr)

        # The probability to go right
        prob_right = np.nan_to_num(np.roll(arr, -1, axis=1)/arr)

        # The probability to go left
        prob_left = np.nan_to_num(np.roll(arr, 1, axis=1)/arr)

        start = 0
        zero_one = np.array([0, 1])
        p_m = np.array([1, -1])
        temp = np.where(current_state == 1)

        for i in range(n):
            direction = np.random.choice(p_m)
            if not even:
                start = np.random.choice(zero_one)

            temp_left = prob_left[temp]
            temp_right = prob_right[temp]

            if not even:
                start = np.random.choice(zero_one)

            if direction == -1:
                probs = (temp_left[start:-1:2] *
                         np.roll(temp_right, 1, axis=0)[start:-1:2])
            else:
                probs = temp_right[start:-1:2]*temp_left[start+1::2]

            r_nums = np.random.random(choices)
            success = r_nums < probs

            for j in np.where(success)[0]:
                idx = j*2+start
                temp_state = current_state[:, [idx+direction, idx]]
                current_state[:, [idx, idx+direction]] = temp_state
                temp_state_2 = temp[0][[idx+direction, idx]]
                temp[0][[idx, idx+direction]] = temp_state_2

            out += current_state

        return out/(n+1)

    def fast_glynn_perm(self, M):
        def cmp(a, b):
            if a == b:
                return 0
            elif a > b:
                return 1
            else:
                return -1
        row_comb = np.sum(M, axis=0, dtype='float128')
        n = len(M)

        total = 0
        old_grey = 0
        sign = +1

        binary_power_dict = {2**i: i for i in range(n)}
        num_loops = 2**(n-1)

        for bin_index in range(1, num_loops + 1):
            total += sign * np.multiply.reduce(row_comb)

            new_grey = bin_index ^ (bin_index // 2)
            grey_diff = old_grey ^ new_grey
            grey_diff_index = binary_power_dict[grey_diff]
            direction = 2 * cmp(old_grey, new_grey)
            if direction:
                new_vector = M[grey_diff_index]
                row_comb += new_vector * direction

            sign = -sign
            old_grey = new_grey

        return total//num_loops
    
    
def calc_cv_vector(path, interfaces):                                           
    path_max, _ = path.ordermax                                                 
    value = 1                                                                   
    return tuple(value if i <= path_max else 0 for i in interfaces)         
